## Example: SpecsScan loading
This is an example showcasing the loading of trARPES data as collected using the Phoibos detector at FHI Berlin.

The band dispersion is loaded as a xarray dataframe demonstrating different modes of operation 

First, the SpecsScan class is imported which has the scan loader as its class method.

In [ ]:
%load_ext autoreload
%autoreload 2
from specsscan import SpecsScan
import matplotlib.pyplot as plt
import numpy as np
%matplotlib widget

Here, a SpecsScan class instance is created as per the configuration provided in [config.yaml](../tests/data/config.yaml). The user may set the entries in config.yaml file, for example, the data path and conversion parameters as per the requirements before creating this instance.

In addition to the provided config files, config files from different locations are optionally included as well (see documentation).

In [ ]:
sps = SpecsScan(config="../tests/data/config.yaml")

### Loading data

The load_scan method loads the scan as an xarray of the data converted into angular/energy coordinates along with the metadata of the scan.

In [ ]:
path = "../tests/data/" # Path to the test data set
# The path may be changed to point to the scan folder of the data of interest (for example, on a server drive)
res_xarray = sps.load_scan(
    scan=4450, # Scan number for an example mirror scan
    path = path,
)

The data are from a mirror scan, showing the mirror position as third dimension:

In [ ]:
res_xarray.dims

We can plot, e.g., selected steps of the scan:

In [ ]:
plt.figure()
res_xarray[:,:,0].plot()

### Cropping data

The image contains data beyond the boundaries given by the illuminated area of the MCP, which should be removed. For this, the ``crop`` option of the converter can be used:

In [ ]:
res_xarray = sps.load_scan(
    scan=4450, # Scan number for an example mirror scan
    path = path,
    crop=True,
)

The loader has given a warning saying that the cropping parameters do not exist yet. Therefore, an interactive cropping tool can be used to crop the data while also saving the crop ranges into a class attribute for later scans. Pressing ``crop`` applies the cropping to the test image, and stores the cropping information in the class.

One can provide relative cropping ranges either as keyword parameters, or in the config file, and optionally directly apply the settings to make the tool non-interactive.

In [ ]:
sps.crop_tool(
    ek_range_min=0.08,
    ek_range_max=0.88,
    ang_range_min=0.15,
    ang_range_max=0.85,
    apply=True,
)

Load the scan again to apply it to all images:

In [ ]:
res_xarray = sps.load_scan(
    scan=4450, # Scan number for an example mirror scan
    path = path,
)

We can e.g. also get a plot along the third dimension, by intergrating along the first.

One can also access the conversion result from a class accessor:

In [ ]:
plt.figure()
sps.result.loc[{"Angle": slice(-5, 5)}].sum(axis=0).plot()

The metadata associated with the scan is added as an attribute to the xarray

In [ ]:
sps.result.attrs["metadata"].keys()

### Loading with selected iterations

3D scans, where the images are recorded as a function of a third parameter (generally delay or in this case, mirrorX), can also be loaded with an option to average only the given iterations passed as a list or slice object. 

In [ ]:
plt.figure()
sps.load_scan(
    scan=4450,
    path=path,
    iterations=np.s_[0, 1:2],
).sum(axis=2).plot()

Another useful functionality is to load a 3D scan as a function of iterations averaged over the scan parameter (in this case, mirrorX). This is done using the check_scan method

In [ ]:
res_xarray_check = sps.check_scan(
    scan=4450,
    delays=0, # for a fixed delay of index, 1
    path=path,
)
plt.figure()
res_xarray_check.loc[{"Angle": slice(-5, 5)}].sum(axis=(0)).plot()

### Saving
Data can be saved, e.g., as hdf5 files including metadata for convenient processing in other notebooks or software.

In [ ]:
sps.load_scan(
    scan=4450, # Scan number for an example mirror scan
    path = path,
)
sps.save("example_data.h5")